In [12]:
%load_ext autoreload
%autoreload 2

import os
import sys
sys.path.insert(0, '..')

from metrics.charades_classify import *

In [2]:
gt_path = '/data/Datasets/Charades/Annotations/Charades_v1_test.csv'
data_path ='/data/Datasets/Charades/Charades_v1_480/'

# gt_ids, gt_classes = load_causal_charades(gt_path, data_path)

In [6]:
gt_ids = []
gt_targets = []
with open(gt_path) as f:
    reader = list(csv.DictReader(f))

count = 0
offset = 0
total = len(reader)
for ir, row in enumerate(reader):
    with tqdm(desc='Video {}/{} ({:.02%})'.format(ir, total, ir/total), total=total,
              leave=False, maxinterval=3600) as t:
        vid = row['id']
        actions = row['actions']
        if actions == '':
            actions = []
        else:
            actions = [a.split(' ') for a in actions.split(';')]
            actions = [{
                'class': x,
                'start': float(y),
                'end': float(z)
            } for x, y, z in actions]
        video_stream = CharadesStream(os.path.join(data_path, vid+'.mp4'), actions,
                                      clip_length=0)
        gt_ids.append(video_stream.target['video_path'])
        gt_targets.append(video_stream.target['target'])

        if (count - offset) >= total // 20:  # Update progressbar every 5%
            t.update(count - offset)
            offset = count

In [24]:
gt_ids2 = np.hstack(gt_ids)
gt_targets2 = np.vstack(gt_targets)

In [25]:
gt_ids2.shape, gt_targets2.shape

((1284224,), (1284224, 157))

In [26]:
gt_clip_causal_path = '/data/Datasets/Charades/Annotations/Charades_v1_causal_allFrames_test.txt'
text = ''
for f_id, f_classes in zip(gt_ids2, gt_targets2):
    text += '{} {}\n'.format(f_id, np.array2string(f_classes, separator=' ')[1:-1].replace('\n', ''))
    
with open(gt_clip_causal_path, 'w') as file:
    file.write(text)

In [5]:
len(gt_ids)

1153408

In [3]:
gt_ids_clip, gt_classes_clip = read_file('/data/Datasets/Charades/Annotations/Charades_v1_causal_clip_test.txt')
gt_ids_frame, gt_classes_frame = read_file('/data/Datasets/Charades/Annotations/Charades_v1_causal_lastFrame_test.txt')

In [4]:
np.all(gt_classes_clip == gt_classes_frame)

False